In [24]:
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import os
import sys
sys.path.append("../")
import random
from aif360.algorithms.inprocessing.gerryfair_classifier import *
from aif360.algorithms.inprocessing.gerryfair.clean import *
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import *
from aif360.algorithms.inprocessing.gerryfair.auditor import *
from sklearn import svm
from sklearn import tree
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
import pickle
import matplotlib.pyplot as plt



We first demonstrate how to instantiate a `Model`, `train` it with respect to rich subgroup fairness, and `predict` the label of a new example. We remark that when we set the `print_flag = True` at each iteration of the algorithm we print the error, fairness violation, and violated group size of most recent model. The error is the classification error of the classifier. At each round the Learner tries to find a classifier that minimizes the classification error plus a weighted sum of the fairness disparities on all the groups that the Auditor has found up until that point. By contrast the Auditor tries to find the group at each round with the greatest rich subgroup disparity with respect to the Learner's model. We define `violated group size` as the size (as a fraction of the dataset size) of this group, and the `fairness violation` as the `violated group size` times the difference in the statistical rate (FP or FN rate) on the group vs. the whole population. 

In the example below we set `max_iterations=5000` which is in-line with time to convergence observed in [the rich subgroup fairness empirical paper](https://arxiv.org/abs/1808.08166), but advise that this can be highly dataset dependent. Our target $\gamma$-disparity is `gamma = .001`, our statistical rate is false positive rate or `FP`, and our weighted regression oracle is linear regression. We observe that the unconstrained (with no fairness constraint) classifier has error $.185$ and $\gamma$-disparity $.022$. After $5000$ iterations we obtain a classifier that $\gamma$-fair, and has error $.230$. We note that we converge to a fair, and almost as accurate classifier after just a hundred iterations.


In [26]:
C = 10
print_flag = True
gamma = .001
max_iterations = 5000

fair_model = Model(C=C, printflag=print_flag, gamma=gamma, fairness_def='FP',
             max_iters=max_iterations, heatmapflag=False)
data_set = load_preproc_data_adult(sub_samp=200)

# fit method
communities_all_errors, communities_violations = fair_model.fit(data_set,
                                                 early_termination=True, return_values=True)
# predict method
dataset_yhat = fair_model.predict(data_set)



iteration: 1
most accurate classifier error: 0.185, most accurate class unfairness: 0.02205882352941177, violated group size: 0.39
iteration: 2
error: 0.21, fairness violation: 0.011029411764705885, violated group size: 0.39
iteration: 3
error: 0.21833333333333335, fairness violation: 0.007352941176470589, violated group size: 0.39
iteration: 4
error: 0.2225, fairness violation: 0.005514705882352942, violated group size: 0.39
iteration: 5
error: 0.225, fairness violation: 0.004411764705882354, violated group size: 0.39
iteration: 6
error: 0.22666666666666666, fairness violation: 0.0036764705882352962, violated group size: 0.39
iteration: 7
error: 0.22785714285714284, fairness violation: 0.003151260504201681, violated group size: 0.375
iteration: 8
error: 0.22875, fairness violation: 0.002757352941176472, violated group size: 0.39
iteration: 9
error: 0.22944444444444442, fairness violation: 0.0024509803921568627, violated group size: 0.375
iteration: 10
error: 0.23, fairness violation: 

iteration: 77
error: 0.23071428571428573, fairness violation: 0.0011777438248026485, violated group size: 0.375
iteration: 78
error: 0.23051282051282052, fairness violation: 0.0012254901960784314, violated group size: 0.375
iteration: 79
error: 0.23031645569620252, fairness violation: 0.0012720277984611568, violated group size: 0.375
iteration: 80
error: 0.23037500000000002, fairness violation: 0.0012561274509803925, violated group size: 0.375
iteration: 81
error: 0.23043209876543208, fairness violation: 0.0012406197046719927, violated group size: 0.39
iteration: 82
error: 0.23048780487804876, fairness violation: 0.0012254901960784318, violated group size: 0.39
iteration: 83
error: 0.23054216867469882, fairness violation: 0.001210725253957005, violated group size: 0.375
iteration: 84
error: 0.2305952380952381, fairness violation: 0.0011963118580765647, violated group size: 0.39
iteration: 85
error: 0.23064705882352943, fairness violation: 0.0011822376009227229, violated group size: 0.3

iteration: 151
error: 0.2310927152317881, fairness violation: 0.0010225944682508768, violated group size: 0.375
iteration: 152
error: 0.23098684210526316, fairness violation: 0.0010481166150670806, violated group size: 0.39
iteration: 153
error: 0.23088235294117646, fairness violation: 0.0010733051390490842, violated group size: 0.39
iteration: 154
error: 0.23077922077922075, fairness violation: 0.0010981665393430104, violated group size: 0.375
iteration: 155
error: 0.2306774193548387, fairness violation: 0.0011227071473750795, violated group size: 0.375
iteration: 156
error: 0.23057692307692307, fairness violation: 0.0011469331322272506, violated group size: 0.39
iteration: 157
error: 0.23060509554140127, fairness violation: 0.0011396278256525544, violated group size: 0.375
iteration: 158
error: 0.23063291139240505, fairness violation: 0.0011324149913129811, violated group size: 0.375
iteration: 159
error: 0.23066037735849057, fairness violation: 0.0011252928844493776, violated group 

iteration: 226
error: 0.2314159292035398, fairness violation: 0.0009218289085545729, violated group size: 0.375
iteration: 227
error: 0.23134361233480177, fairness violation: 0.0009393625291526308, violated group size: 0.39
iteration: 228
error: 0.2312719298245614, fairness violation: 0.0009567423460612323, violated group size: 0.39
iteration: 229
error: 0.23120087336244538, fairness violation: 0.000973970374175872, violated group size: 0.39
iteration: 230
error: 0.2311304347826087, fairness violation: 0.0009910485933503845, violated group size: 0.375
iteration: 231
error: 0.23106060606060608, fairness violation: 0.0010079789491554204, violated group size: 0.375
iteration: 232
error: 0.23099137931034483, fairness violation: 0.00102476335361731, violated group size: 0.39
iteration: 233
error: 0.2309227467811159, fairness violation: 0.0010414036859378954, violated group size: 0.375
iteration: 234
error: 0.23085470085470086, fairness violation: 0.001057901793195912, violated group size: 0

iteration: 300
error: 0.2314333333333333, fairness violation: 0.0009068627450980403, violated group size: 0.39
iteration: 301
error: 0.2313787375415282, fairness violation: 0.0009201354960588894, violated group size: 0.375
iteration: 302
error: 0.23132450331125828, fairness violation: 0.0009333203480067529, violated group size: 0.39
iteration: 303
error: 0.2312706270627063, fairness violation: 0.0009464181712288881, violated group size: 0.375
iteration: 304
error: 0.23121710526315786, fairness violation: 0.0009594298245614041, violated group size: 0.375
iteration: 305
error: 0.2311639344262295, fairness violation: 0.0009723561555769856, violated group size: 0.39
iteration: 306
error: 0.2311111111111111, fairness violation: 0.0009851980007689358, violated group size: 0.375
iteration: 307
error: 0.2310586319218241, fairness violation: 0.000997956185731622, violated group size: 0.375
iteration: 308
error: 0.2310064935064935, fairness violation: 0.0010106315253374085, violated group size: 

...

iteration: 4641
error: 0.23092975651799183, fairness violation: 0.0009997211554304952, violated group size: 0.375
iteration: 4642
error: 0.23093063334769495, fairness violation: 0.0009995057911143749, violated group size: 0.375
iteration: 4643
error: 0.23093150979969848, fairness violation: 0.0009992905195677212, violated group size: 0.39
iteration: 4644
error: 0.23093238587424636, fairness violation: 0.0009990753407306048, violated group size: 0.375
iteration: 4645
error: 0.23093326157158237, fairness violation: 0.0009988602545431494, violated group size: 0.375
iteration: 4646
error: 0.2309341368919501, fairness violation: 0.0009986452609455293, violated group size: 0.375
iteration: 4647
error: 0.23093501183559284, fairness violation: 0.0009984303598779705, violated group size: 0.375
iteration: 4648
error: 0.2309358864027539, fairness violation: 0.000998215551280751, violated group size: 0.39
iteration: 4649
error: 0.23093245859324585, fairness violation: 0.0009990552469643459, violat

iteration: 4714
error: 0.2309291472210437, fairness violation: 0.0009998377798297914, violated group size: 0.39
iteration: 4715
error: 0.2309300106044539, fairness violation: 0.000999625725157505, violated group size: 0.39
iteration: 4716
error: 0.23093087362171333, fairness violation: 0.0009994137604151052, violated group size: 0.39
iteration: 4717
error: 0.23093173627305488, fairness violation: 0.000999201885545397, violated group size: 0.39
iteration: 4718
error: 0.23093259855871132, fairness violation: 0.0009989901004912331, violated group size: 0.39
iteration: 4719
error: 0.230933460478915, fairness violation: 0.0009987784051955152, violated group size: 0.375
iteration: 4720
error: 0.2309343220338983, fairness violation: 0.000998566799601194, violated group size: 0.375
iteration: 4721
error: 0.23093518322389325, fairness violation: 0.0009983552836512683, violated group size: 0.39
iteration: 4722
error: 0.23093604404913173, fairness violation: 0.000998143857288784, violated group s

iteration: 4788
error: 0.23092522974101923, fairness violation: 0.00100076580339738, violated group size: 0.375
iteration: 4789
error: 0.23092608060137818, fairness violation: 0.001000556831628034, violated group size: 0.39
iteration: 4790
error: 0.23092693110647183, fairness violation: 0.0010003479471120365, violated group size: 0.39
iteration: 4791
error: 0.23092778125652266, fairness violation: 0.0010001391497947514, violated group size: 0.375
iteration: 4792
error: 0.2309286310517529, fairness violation: 0.0009999304396215891, violated group size: 0.39
iteration: 4793
error: 0.23092948049238474, fairness violation: 0.0009997218165380042, violated group size: 0.375
iteration: 4794
error: 0.23093032957863996, fairness violation: 0.0009995132804894982, violated group size: 0.39
iteration: 4795
error: 0.2309311783107404, fairness violation: 0.0009993048314216172, violated group size: 0.375
iteration: 4796
error: 0.23093202668890744, fairness violation: 0.0009990964692799529, violated g

iteration: 4862
error: 0.23093377211024269, fairness violation: 0.0009986409207862472, violated group size: 0.39
iteration: 4863
error: 0.23093460826650214, fairness violation: 0.0009984355658775924, violated group size: 0.375
iteration: 4864
error: 0.2309354440789474, fairness violation: 0.0009982302954076342, violated group size: 0.375
iteration: 4865
error: 0.23093627954779034, fairness violation: 0.000998025109324303, violated group size: 0.39
iteration: 4866
error: 0.2309371146732429, fairness violation: 0.0009978200075755723, violated group size: 0.39
iteration: 4867
error: 0.2309379494555168, fairness violation: 0.0009976149901094581, violated group size: 0.375
iteration: 4868
error: 0.23093878389482334, fairness violation: 0.000997410056874021, violated group size: 0.39
iteration: 4869
error: 0.2309355103717396, fairness violation: 0.0009982119773356026, violated group size: 0.39
iteration: 4870
error: 0.23093223819301847, fairness violation: 0.0009990135684663983, violated gro

iteration: 4936
error: 0.23093395461912483, fairness violation: 0.000998565957987731, violated group size: 0.375
iteration: 4937
error: 0.23093072716224428, fairness violation: 0.0009993565990301305, violated group size: 0.39
iteration: 4938
error: 0.23092750101255569, fairness violation: 0.0010001469198452951, violated group size: 0.39
iteration: 4939
error: 0.230924276169265, fairness violation: 0.0010009369206277346, violated group size: 0.39
iteration: 4940
error: 0.23092105263157897, fairness violation: 0.0010017266015718003, violated group size: 0.375
iteration: 4941
error: 0.23092187816231532, fairness violation: 0.0010015238639475195, violated group size: 0.375
iteration: 4942
error: 0.230922703358964, fairness violation: 0.0010013212083700312, violated group size: 0.375
iteration: 4943
error: 0.2309235282217277, fairness violation: 0.0010011186347895398, violated group size: 0.39
iteration: 4944
error: 0.23092435275080908, fairness violation: 0.0010009161431562893, violated gr

In [ ]:

# output heatmap (brute force)
# replace None with the relative path if you want to save the plot
fair_model.heatmapflag = True
fair_model.save_heatmap(fair_model.max_iters, data_set, dataset_yhat.labels, None, None)






In [ ]:
# auditing a classifier for unfairness
# instantiate auditor
auditor = Auditor(data_set, 'FP')
group = auditor.get_group(dataset_yhat.labels, auditor.get_baseline(array_to_tuple(data_set.labels), array_to_tuple(dataset_yhat.labels)))
print('gamma disparity: {}'.format(group.weighted_disparity))






In [32]:
# run & create pareto curves
fair_model.max_iters = 10
fair_model.printflag = False
gamma_list = [.01, .02, .03, 1.0]
fair_model.pareto(data_set, gamma_list)



iteration: 1
most accurate classifier error: 0.185, most accurate class unfairness: 0.02205882352941177, violated group size: 0.39
iteration: 2
error: 0.21, fairness violation: 0.011029411764705885, violated group size: 0.39
iteration: 3
error: 0.21833333333333335, fairness violation: 0.007352941176470589, violated group size: 0.39
iteration: 4
error: 0.2225, fairness violation: 0.005514705882352942, violated group size: 0.39
iteration: 5
error: 0.225, fairness violation: 0.004411764705882354, violated group size: 0.39
iteration: 6
error: 0.22666666666666666, fairness violation: 0.0036764705882352962, violated group size: 0.39
iteration: 7
error: 0.225, fairness violation: 0.0038515406162464992, violated group size: 0.375
iteration: 8
error: 0.22375, fairness violation: 0.0039828431372549026, violated group size: 0.375
iteration: 9
error: 0.2227777777777778, fairness violation: 0.004084967320261439, violated group size: 0.39
iteration: 1
most accurate classifier error: 0.185, most accu

([0.2227777777777778, 0.21055555555555558, 0.185, 0.185],
 [0.004084967320261439,
  0.0062636165577342065,
  0.02205882352941177,
  0.02205882352941177],
 [0.0, 0.0057446808510638455, 0.018191489361702128, 0.018191489361702128])

In [ ]:
def multiple_classifiers_pareto(dataset, gamma_list=[0.002, 0.005, 0.01, 0.02, 0.05, 0.1], save_results=False):

    ln_predictor = linear_model.LinearRegression()
    svm_predictor = svm.LinearSVR()
    tree_predictor = tree.DecisionTreeRegressor(max_depth=3)
    kernel_predictor = KernelRidge(alpha=1.0, gamma=1.0, kernel='rbf')
    predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': 10},
                      'SVR': {'predictor': svm_predictor, 'iters': 10},
                      'Tree': {'predictor': tree_predictor, 'iters': 10},
                      'Kernel': {'predictor': kernel_predictor, 'iters': 10}}

    results_dict = {}

    for pred in predictor_dict:
        print('Curr Predictor: {}'.format(pred))
        predictor = predictor_dict[pred]['predictor']
        max_iters = predictor_dict[pred]['iters']
        fair_clf = Model(C=100, printflag=True, gamma=1, predictor=predictor, max_iters=max_iters)
        fair_clf.set_options(max_iters=max_iters)
        errors, fp_violations, fn_violations = fair_clf.pareto(dataset, gamma_list)
        results_dict[pred] = {'errors': errors, 'fp_violations': fp_violations, 'fn_violations': fn_violations}
        plt.plot(errors, fp_violations, label=pred)

    if save_results:
        pickle.dump(results_dict, open('results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb'))

    plt.xlabel('Error')
    plt.ylabel('Unfairness')
    plt.legend()
    plt.title('Error vs. Unfairness\n(Communities & Crime Dataset)')
    plt.show()
    
    
multiple_classifiers_pareto(data_set)

In [ ]:

def fp_vs_fn(dataset, fair_model, gamma_list):
    fp_auditor = Auditor(dataset, 'FP')
    fn_auditor = Auditor(dataset, 'FN')
    fp_violations = []
    fn_violations = []
    for g in gamma_list:
        fair_model.set_options(gamma=g)
        fair_model.fit(dataset)
        predictions = (fair_model.predict(dataset)).labels
        predictions_inv = [abs(1 - p) for p in predictions]
        _, fp_diff = fp_auditor.audit(predictions)
        _, fn_diff = fn_auditor.audit(predictions_inv)
        fp_violations.append(fp_diff)
        fn_violations.append(fn_diff)

    print((fp_violations, fn_violations))

    plt.plot(fp_violations, fn_violations, label='communities')
    plt.xlabel('False Positive Disparity')
    plt.ylabel('False Negative Disparity')
    plt.legend()
    plt.title('FP vs FN Unfairness')
    plt.show()
    
C = 10
print_flag = True
gamma = .01
max_iterations = 10
fair_def = 'FP'
fair_model = Model(C=C, printflag=print_flag, gamma=gamma, fairness_def=fair_def,
             max_iters=max_iterations)
data_set = load_preproc_data_adult(sub_samp=50)
gamma_list = [0.001, 0.002, 0.003, 0.004, 0.005, 0.0075, 0.01, 0.02, 0.03, 0.05]
fp_vs_fn(data_set, fair_model, gamma_list)

NameError: name 'load_preproc_data_adult' is not defined